In [179]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import tqdm

In [180]:
base_url = "https://pedia.watcha.com/ko-KR/?domain=movie"
driver = webdriver.Chrome("./chromedriver")

In [183]:
driver.get(base_url)
driver.implicitly_wait(20)

In [184]:
rankings = driver.find_elements_by_class_name('css-8y23cj')
for i in tqdm.tqdm(range(11, len(rankings))):
    # 영화 상세 페이지
    ranking_xPath = '//*[@id="root"]/div/div[1]/section/div/section/div[1]/div[2]/div/div[1]/div/div/ul/li['+str(i)+']'
    ranking = driver.find_element(By.XPATH, ranking_xPath)
    ranking.click()
    # 코멘트 페이지
    comment_xPath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div/div/div[1]/div[1]/div/div/section[5]/div[1]/div/header/div/div/a'
    comment = driver.find_element(By.XPATH, comment_xPath)
    comment.click()


    break

  0%|          | 0/119 [00:02<?, ?it/s]


In [185]:
# 스크롤 끝까지 내려서 다음 정보 얻기
t_end = time.time() + 1800
while time.time() < t_end:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-5);")
    time.sleep(2)

In [186]:
# 코멘트 페이지에 있는 유저 페이지 목록 수집
users = driver.find_elements_by_class_name('css-bawlbm')
users_href = []
for i in tqdm.tqdm(range(1, len(users))):
    user_xPath = '//*[@id="root"]/div/div[1]/section/section/div/div/div/ul/div['+str(i)+']/div[1]/div[1]/a'
    user = driver.find_element(By.XPATH, user_xPath)
    users_href.append(user.get_attribute('href'))
print(len(users_href))

100%|██████████| 2633/2633 [01:40<00:00, 26.18it/s]

2633


In [187]:
for user_href in tqdm.tqdm(users_href):
    # 유저 분석 탭 이동
    last_tap = driver.window_handles[-1]
    driver.switch_to_window(window_name=last_tap)
    driver.get(user_href)
    analysis_xPath = '//*[@id="root"]/div/div[1]/section/div/div/div/div/section/div[2]/div[1]/ul/li/a/span[2]'
    analysis = driver.find_element(By.XPATH, analysis_xPath)
    analysis.click()

    # 유저의 장르별 점수 수집 후 한 명당 하나의 csv 파일로 저장
    last_tap = driver.window_handles[-1]
    driver.switch_to_window(window_name=last_tap)
    user_name_xPath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div[1]/header/div/div/h1/div/div[2]'
    user_name = driver.find_element(By.XPATH, user_name_xPath).text

    try:
        more_xPath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div[3]/section[5]/div[2]/div/button'
        more = driver.find_element(By.XPATH, more_xPath)
        more.click()

        top_3_ul_xPath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div[3]/section[5]/div[2]/div/div/ul'
        other_genre_ul_xPATH = '//*[@id="root"]/div/div[1]/section/div/div[2]/div[3]/section[5]/div[2]/div/ul'
        top_3_ul = driver.find_element(By.XPATH, top_3_ul_xPath)
        other_genre_ul = driver.find_element(By.XPATH, other_genre_ul_xPATH)

        genre = {}
        for top_3_div in top_3_ul.find_elements_by_css_selector('li'):
            genre_name = top_3_div.find_element_by_css_selector('div').text
            genre_score = top_3_div.find_element_by_css_selector('div+div').text
            genre[genre_name] =  int(genre_score.split('점')[0])

        for other_genre_div in other_genre_ul.find_elements_by_css_selector('li'):
            genre_name = other_genre_div.text
            genre_score = other_genre_div.find_element_by_css_selector('span').text
            genre[genre_name.split('\n')[0]] = int(genre_score.split('점')[0])

        driver.close()

        with open('users/'+user_name+'.csv', 'w') as f:
            w = csv.writer(f)
            w.writerow(genre.keys())
            w.writerow(genre.values())
    except:
        pass

  0%|          | 0/2633 [00:00<?, ?it/s]/var/folders/xr/ktrktz4s3ss6jp1765b9dq4w0000gn/T/ipykernel_75439/1368311777.py:3: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window_name=last_tap)
/var/folders/xr/ktrktz4s3ss6jp1765b9dq4w0000gn/T/ipykernel_75439/1368311777.py:10: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window_name=last_tap)
100%|██████████| 2633/2633 [9:07:42<00:00, 12.48s/it]  
